In [87]:
import pandas as pd

In [88]:
data=pd.read_excel('marketing_campaign.xlsx')

In [89]:
from sklearn import metrics

In [90]:
data.shape

(5000, 12)

In [91]:
x=data.drop(["Offer_acceptance"],axis=1)
y=data["Offer_acceptance"]


In [92]:
from sklearn.model_selection import train_test_split

In [93]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.3)

In [94]:
from sklearn import tree

In [95]:
ctree=tree.DecisionTreeClassifier()

In [96]:
ctree=ctree.fit(x_train,y_train)

In [97]:
y_pred=ctree.predict(x_test)

In [98]:
from sklearn.model_selection import cross_val_score
cross_val_score(ctree,x,y,cv=5,scoring="accuracy").mean()


0.8126099256099255

In [99]:
from sklearn.model_selection import GridSearchCV

In [100]:
param_grid={"criterion" :["gini","entropy"],
           "max_features" :[1,2],
            "max_depth"    :[1,2,3],
            "splitter"     :["best","random"],
            "min_samples_leaf" :[2,3,4],
            "class_weight":[{0:1,1:2},{0:1,1:4}]    
}

In [101]:
optimize=GridSearchCV(ctree,param_grid,scoring="recall",cv=5)
optimize

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'criterion': ['gini', 'entropy'], 'max_features': [1, 2], 'max_depth': [1, 2, 3], 'splitter': ['best', 'random'], 'min_samples_leaf': [2, 3, 4], 'class_weight': [{0: 1, 1: 2}, {0: 1, 1: 4}]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='recall', verbose=0)

In [102]:
optimize.fit(x_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'criterion': ['gini', 'entropy'], 'max_features': [1, 2], 'max_depth': [1, 2, 3], 'splitter': ['best', 'random'], 'min_samples_leaf': [2, 3, 4], 'class_weight': [{0: 1, 1: 2}, {0: 1, 1: 4}]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='recall', verbose=0)

In [103]:
optimize.best_score_

0.5460426040494938

In [104]:

from sklearn.metrics import confusion_matrix

In [105]:
print(confusion_matrix(y_test,y_pred))

[[1086  140]
 [ 118  156]]


In [106]:
metrics.recall_score(y_test,y_pred)

0.5693430656934306

In [107]:
optimize.best_params_

{'class_weight': {0: 1, 1: 4},
 'criterion': 'entropy',
 'max_depth': 3,
 'max_features': 2,
 'min_samples_leaf': 2,
 'splitter': 'random'}

In [108]:
optimize.best_estimator_

DecisionTreeClassifier(class_weight={0: 1, 1: 4}, criterion='entropy',
            max_depth=3, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='random')

In [109]:
ctree_final = optimize.best_estimator_

In [110]:
ctree_final

DecisionTreeClassifier(class_weight={0: 1, 1: 4}, criterion='entropy',
            max_depth=3, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='random')

In [111]:
y_pred = ctree_final.predict(x_test)
print('Accuracy', metrics.accuracy_score(y_pred, y_test))

Accuracy 0.6993333333333334


In [112]:
metrics.precision_score(y_pred,y_test)

0.572992700729927

In [113]:
imp_df = pd.DataFrame()
imp_df['col_names']=x.columns
imp_df['Importance']= ctree_final.feature_importances_*100
imp_df

,col_names,Importance
0,Age,0.000000
1,Experience,1.008176
2,Income,1.065690
3,Family,0.000000
4,CCAvg,54.406731
5,Education,14.881396
6,Mortgage,5.341569
7,Securities Account,0.000000
8,CD Account,23.296438
9,Online,0.000000


In [114]:
imp_df = pd.DataFrame()
imp_df['col_names']=x.columns
imp_df['Importance']= ctree_final.feature_importances_*100
imp_df.sort_values('Importance', ascending=False)

,col_names,Importance
4,CCAvg,54.406731
8,CD Account,23.296438
5,Education,14.881396
6,Mortgage,5.341569
2,Income,1.065690
1,Experience,1.008176
0,Age,0.000000
3,Family,0.000000
7,Securities Account,0.000000
9,Online,0.000000


In [115]:
import pickle
pickle.dump(ctree_final, open('final_ctree_model_dec11.pk','wb'))